In [4]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
R.<x> = PolynomialRing(QQ)
Scplx= PowerSeriesRing(CC,'x',default_prec=300)

def division(dividend, divisor) : 
    quotient=(dividend._maxima_().divide(divisor).sage())[0] 
    remainder=(dividend._maxima_().divide(divisor).sage())[1] 
    return (quotient,remainder)

def polynomialdivision(dividend, divisor):
    q,r = dividend.maxima_methods().divide(divisor)
    return (q,r)

def ordP(n,p):
    fct=factor(n)
    lf=len(fct)
    for k in [0..lf-1]:
        prime=fct[k][0]
        if prime==p:
            answer=fct[k][1]
            return answer
    return 0
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def exNo3cX(n,m,x):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.

def JX(n,m,x):
    exn = exNo3cX(n+2,m,x)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.


def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def normalJX(n,m,x):
    f = JX(n,m,x)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def JXcoefficient(n,m,x):
    f = JX(n+2,m,x)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNewStep(n,m): 
    f=J(n,m)
    fl=f.list()
    return (sum(c*x**i*(m**3*2**6)**i for i,c in enumerate(fl)))

def xjNew(n,m):
    return xjNewStep(n-1,m)

def xjNewCoefficient(n,m):
    if n>0:return (xjNew(n,m)-xjNew(n-1,m))/x^n
    if n==0:return 1
    
def xjNewTest(n,m):
    return D(xjNewStep(n-1,m))

def j(n,m):return xjNew(n,m)/x

def H4(n,m):
    jay=j(n+1,m)
    djay=x*derivative(jay,x) # bc variable is tau, not x; chain rule.
    numerator = djay^2
    denominator = jay*(jay-2^6*m^3)
    return ((S(numerator/denominator)^(1/(m-2))).O(n+1)).polynomial()

def H6(n,m): #different than in draft 4mar21 and Mathematica.
    def base6(n,m):
        jay=j(n+1,m)
        num=(x*derivative(jay,x))^m # chain rule; variable is tau not x.
        den = expand(jay^(m-1)*(j(n+1,m)-2^6*m^3))
        return L((-1)^m*(num/den)).O(n+1)
    return (S(base6(n,m)^(1/(m-2))).polynomial())

def deltaInfty(n,m): 
    # definition 9.1 in the paper to power (m-2)
    # so, per remark 5 in paper v. 28jul21,
    # the weight is 12 for each m.
    jay=J(n,m)
    djay=x*derivative(jay,x)
    numerator=djay^(2*m)
    denominator=jay^(2*m-2)*(jay-1)^m
    answer=S((numerator/denominator)^(1/(m-2)))
    return answer.polynomial()
            
def deltaInftyStrike(n,m): 
    # substitute 2^6*m^3*x for x in def 9.1; not
    # yet defined in draft  21h28jul21
    f=deltaInfty(n,m)
    fl=f.list()
    df=f.degree()
    sm=sum(fl[k]*(x*m^3*2^6)^k for k in [0..df])
    dsm=sm.degree()
    sml=sm.list()
    notzero=[]
    for entry in sml:
        if (entry!=0):notzero=notzero+[entry]
    leadingterm=notzero[0]
    sm2=sum(fl[k]*(x*m^3*2^6)^k/leadingterm for k in [0..dsm])
    return sm2
    
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})
def deltaDiamond(n,m):
    return D(S(expand((H4(n+2,m)^3)/j(n+2,m)))).truncate(n+1)
def nieburTau(n):
# The contortions below sidestep mysterious (to me) error messages...
# nieburTau is an algorith for Ramanujan''s tau from the paper
# "A formula for Ramanujan's tau-function" by Douglas Niebur.
# (He spells tau with the Greek letter.)
    def step1(n,k):
        return 35*k^4-52*k^3*n+18*k^2*n^2
    def step2(n,k):
        return sigma(k)*sigma(n-k)
    def step3(n):
        ans=0
        for k in [1..n-1]:
            ans=ans+step1(n,k)*step2(n,k)
        return 24*ans
    return n^4*sigma(n)-step3(n)
def deltaDagger(n,m):
    def ddstep1(n,m):
        return D(S(expand((H4(n+2,m)^3)))).truncate(n+1)
    def ddstep2(n,m):
        return D(S((H6(n+2,m)^2))).truncate(n+1)
    return (ddstep1(n,m)-ddstep2(n,m))/1728
def last(list):return list[-1]


def polynomialExponentList(polynomial):
    pel=[]
    fp=polynomial.factor()
    lf=len(fp)
    for n in [0..lf-1]:
        part = fp[n]
        (base,exponent) = part
        pel=pel+[exponent]
    return pel

In [6]:
poly=D(deltaInftyStrike(100,3))
data=[]
ls=poly.list()
for n in [1..len(ls)-1]:
    data=data+[polynomialCoefficient(n,poly)-nieburTau(n)]
print data

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [41]:
for m in [3..6]:
    print "-----------------------------------------------------------------------"
    print  (m,deltaInfty(5,m))
    print
    print (m,deltaInftyStrike(5,m))


-----------------------------------------------------------------------
(3, -2093/3327916660110655488*x^7 - 7/17832200896512*x^6 + 805/1486016741376*x^5 - 23/80621568*x^4 + 7/82944*x^3 - 1/72*x^2 + x)

(3, -16744*x^7 - 6048*x^6 + 4830*x^5 - 1472*x^4 + 252*x^3 - 24*x^2 + x)
-----------------------------------------------------------------------
(4, 127/35184372088832*x^7 - 3/34359738368*x^6 - 105/2147483648*x^5 + 1/262144*x^4 + 3/16384*x^3 - 1/32*x^2 + x)

(4, 17045651456*x^7 - 100663296*x^6 - 13762560*x^5 + 262144*x^4 + 3072*x^3 - 128*x^2 + x)
-----------------------------------------------------------------------
(5, 29685769799/2359296000000000000000*x^7 - 100656411/40960000000000000*x^6 - 9633/3276800000000*x^5 + 961/192000000*x^4 + 279/640000*x^3 - 9/200*x^2 + x)

(5, 29685769799000/9*x^7 - 80525128800*x^6 - 12041250*x^5 + 7688000/3*x^4 + 27900*x^3 - 360*x^2 + x)
-----------------------------------------------------------------------
(6, -5/198359290368*x^7 - 1/272097792*x^6 + 1/22

In [43]:
import time
import pickle
wfile = open('/Users/barrybrent/run12aug21no1.txt','w') 
data=[]
start=time.time()
for m in [3..303]:
    ds=deltaInftyStrike(100,m)
    data=data+[[m,ds]]
    finish=time.time()
    print([m,finish-start])
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

[3, 1.353079080581665]
[4, 2.265439033508301]
[5, 3.9876139163970947]
[6, 4.864794969558716]
[7, 6.770205020904541]
[8, 8.204614877700806]
[9, 10.284827947616577]
[10, 11.642220973968506]
[11, 13.7218599319458]
[12, 15.534689903259277]
[13, 17.727375030517578]
[14, 19.170372009277344]
[15, 21.67025589942932]
[16, 23.371131896972656]
[17, 25.740657091140747]
[18, 27.361300945281982]
[19, 29.657912969589233]
[20, 31.708714962005615]
[21, 34.31066393852234]
[22, 35.88409090042114]
[23, 38.25201106071472]
[24, 40.40206503868103]
[25, 42.976797103881836]
[26, 44.748132944107056]
[27, 47.35822296142578]
[28, 49.55290603637695]
[29, 52.02466702461243]
[30, 53.997610092163086]
[31, 56.4623908996582]
[32, 58.432990074157715]
[33, 61.281373023986816]
[34, 63.19229602813721]
[35, 66.16797995567322]
[36, 68.60910987854004]
[37, 71.29336595535278]
[38, 73.1825590133667]
[39, 76.15023303031921]
[40, 78.60769891738892]
[41, 81.27604603767395]
[42, 83.44093704223633]
[43, 86.10142803192139]
[44, 88.50

In [4]:
import pickle
rfile = open('/Users/barrybrent/run12aug21no1.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
wfile = open('/Users/barrybrent/run13aug21no1.txt','w') 
data=[]
for k in [0..60]:
    data=data+[s[k]]
t = pickle.dumps(str(data))
wfile.write(t)
wfile.close()

In [5]:
import pickle
rfile = open('/Users/barrybrent/run12aug21no1.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
wfile = open('/Users/barrybrent/run13aug21no2.txt','w') 
data=[]
for k in [61..120]:
    data=data+[s[k]]
t = pickle.dumps(str(data))
wfile.write(t)
wfile.close()

In [6]:
import pickle
rfile = open('/Users/barrybrent/run12aug21no1.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
wfile = open('/Users/barrybrent/run13aug21no3.txt','w') 
data=[]
for k in [121..180]:
    data=data+[s[k]]
t = pickle.dumps(str(data))
wfile.write(t)
wfile.close()

In [7]:
import pickle
rfile = open('/Users/barrybrent/run12aug21no1.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
wfile = open('/Users/barrybrent/run13aug21no4.txt','w') 
data=[]
for k in [181..240]:
    data=data+[s[k]]
t = pickle.dumps(str(data))
wfile.write(t)
wfile.close()

In [8]:
import pickle
rfile = open('/Users/barrybrent/run12aug21no1.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
wfile = open('/Users/barrybrent/run13aug21no5.txt','w') 
data=[]
for k in [241..len(s)-1]:
    data=data+[s[k]]
t = pickle.dumps(str(data))
wfile.write(t)
wfile.close()

In [44]:
import pickle
rfile = open('/Users/barrybrent/run12aug21no1.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
test=[]
for n in [0..100]:
    data=[]
    for k in [0..len(s)-1]:
        m = s[k][0]
        poly=s[k][1]
        cf=polynomialCoefficient(n,poly)
        data=data+[(m,cf)]
    rl=R.lagrange_polynomial(data)
    rl=D(rl)
    print (n,rl.degree())
    test=test+[[n,rl]]
wfile = open('/Users/barrybrent/run12aug21no2.txt','w') 
t = pickle.dumps(str(test))
wfile.write(t)
wfile.close()

(0, -1)
(1, 0)
(2, 3)
(3, 6)
(4, 9)
(5, 12)
(6, 15)
(7, 18)
(8, 21)
(9, 24)
(10, 27)
(11, 30)
(12, 33)
(13, 36)
(14, 39)
(15, 42)
(16, 45)
(17, 48)
(18, 51)
(19, 54)
(20, 57)
(21, 60)
(22, 63)
(23, 66)
(24, 69)
(25, 72)
(26, 75)
(27, 78)
(28, 81)
(29, 84)
(30, 87)
(31, 90)
(32, 93)
(33, 96)
(34, 99)
(35, 102)
(36, 105)
(37, 108)
(38, 111)
(39, 114)
(40, 117)
(41, 120)
(42, 123)
(43, 126)
(44, 129)
(45, 132)
(46, 135)
(47, 138)
(48, 141)
(49, 144)
(50, 147)
(51, 150)
(52, 153)
(53, 156)
(54, 159)
(55, 162)
(56, 165)
(57, 168)
(58, 171)
(59, 174)
(60, 177)
(61, 180)
(62, 183)
(63, 186)
(64, 189)
(65, 192)
(66, 195)
(67, 198)
(68, 201)
(69, 204)
(70, 207)
(71, 210)
(72, 213)
(73, 216)
(74, 219)
(75, 222)
(76, 225)
(77, 228)
(78, 231)
(79, 234)
(80, 237)
(81, 240)
(82, 243)
(83, 246)
(84, 249)
(85, 252)
(86, 255)
(87, 258)
(88, 261)
(89, 264)
(90, 267)
(91, 270)
(92, 273)
(93, 276)
(94, 279)
(95, 282)
(96, 285)
(97, 288)
(98, 291)
(99, 294)
(100, 297)


In [9]:
import pickle
rfile = open('/Users/barrybrent/run12aug21no2.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
wfile = open('/Users/barrybrent/run13aug21no6.txt','w') 
data=[]
for k in [0..50]:
    data=data+[s[k]]
t = pickle.dumps(str(data))
wfile.write(t)
wfile.close()

In [11]:
import pickle
rfile = open('/Users/barrybrent/run12aug21no2.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
wfile = open('/Users/barrybrent/run13aug21no7.txt','w') 
data=[]
for k in [51..75]:
    data=data+[s[k]]
t = pickle.dumps(str(data))
wfile.write(t)
wfile.close()

In [12]:
import pickle
rfile = open('/Users/barrybrent/run12aug21no2.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
wfile = open('/Users/barrybrent/run13aug21no8.txt','w') 
data=[]
for k in [76..len(s)-1]:
    data=data+[s[k]]
t = pickle.dumps(str(data))
wfile.write(t)
wfile.close()

In [46]:
import pickle
rfile = open('/Users/barrybrent/run12aug21no2.txt','r')  
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
nfdata=[]
for k in [1..4]:
    n=s[k][0]
    poly=D(s[k][1])
    nf=numericalfactor(poly)
    nfdata=nfdata+[nf]
    print "-----------------------------------------------------------------------"
    print
    print (n,nf)
    print
    print poly.factor()
print nfdata

-----------------------------------------------------------------------

(1, 1)

1
-----------------------------------------------------------------------

(2, -8)

(-8) * x * (x - 2)^2
-----------------------------------------------------------------------

(3, 28)

(28) * (x - 2)^2 * x^2 * (x^2 - 44/7*x + 76/7)
-----------------------------------------------------------------------

(4, -64)

(-64) * (x - 2)^2 * x^3 * (x^4 - 44/3*x^3 + 2096/27*x^2 - 5200/27*x + 5264/27)
[1, -8, 28, -64]


In [47]:
rfile = open('/Users/barrybrent/run12aug21no2.txt','r')  
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
nfdata=[]
for k in [1..len(s)-1]:
    n=s[k][0]
    poly=D(s[k][1])
    nf=numericalfactor(poly)
    nfdata=nfdata+[nf]
print nfdata

[1, -8, 28, -64, 126, -224, 344, -512, 757, -1008, 1332, -1792, 2198, -2752, 3528, -4096, 4914, -6056, 6860, -8064, 9632, -10656, 12168, -14336, 15751, -17584, 20440, -22016, 24390, -28224, 29792, -32768, 37296, -39312, 43344, -48448, 50654, -54880, 61544, -64512, 68922, -77056, 79508, -85248, 95382, -97344, 103824, -114688, 117993, -126008, 137592, -140672, 148878, -163520, 167832, -176128, 192080, -195120, 205380, -225792, 226982, -238336, 260408, -262144, 276948, -298368, 300764, -314496, 340704, -346752, 357912, -387584, 389018, -405232, 441028, -439040, 458208, -492352, 493040, -516096, 551881, -551376, 571788, -616448, 619164, -636064, 682920, -681984, 704970, -763056, 756112, -778752, 834176, -830592, 864360, -917504, 912674, -943944, 1008324, -1008064]


In [51]:
from sage.arith.misc import Sigma
print sigma(3,1);print sigma(3,2)

4
10


In [73]:
def sig3(n):
    summ=0
    dv=divisors(n)
    ln=len(dv)
    for k in [0..ln-1]:
        divisor=dv[k]
        if mod(n/divisor,2)==1:
            summ=summ+divisor^3
    return (-1)^(n+1)*summ

In [77]:
rfile = open('/Users/barrybrent/run12aug21no2.txt','r')  
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..len(s)-1]:
    n=s[k][0]
    poly=D(s[k][1])
    nf=numericalfactor(poly)
    data=data+[nf-sig3(n)]
print data
        

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [2]:
import pickle
rfile = open('/Users/barrybrent/run12aug21no2.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [0..len(s)-1]:
    n=s[k][0]
    poly=D(s[k][1])
    dg=poly.degree()
    test=3*n-3
    if dg!=test:data=data+[n]
print data
    

[0]


In [4]:
import pickle
rfile = open('/Users/barrybrent/run12aug21no2.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..6]:
    n=s[k][0]
    poly=D(s[k][1])
    f=poly.factor()
    print "============================================================================="
    print "n: ",n
    print;print f
   

n:  1

1
n:  2

(-8) * x * (x - 2)^2
n:  3

(28) * (x - 2)^2 * x^2 * (x^2 - 44/7*x + 76/7)
n:  4

(-64) * (x - 2)^2 * x^3 * (x^4 - 44/3*x^3 + 2096/27*x^2 - 5200/27*x + 5264/27)
n:  5

(126) * (x - 2)^2 * x^4 * (x^6 - 4220/189*x^5 + 364300/1701*x^4 - 629344/567*x^3 + 5611696/1701*x^2 - 3141440/567*x + 7226176/1701)
n:  6

(-224) * (x - 2)^2 * x^5 * (x^8 - 3128/105*x^7 + 1078208/2625*x^6 - 25985824/7875*x^5 + 14834912/875*x^4 - 450196352/7875*x^3 + 980173312/7875*x^2 - 432644608/2625*x + 830809856/7875)


In [16]:
import pickle
rfile = open('/Users/barrybrent/run12aug21no2.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [2..5]:
    n=s[k][0]
    poly=s[k][1]
    divideby=(x-2)^2*x^(n-1)
    g=division(poly,divideby)
    print "=========================================================="
    print "n: ",n
    print g
    print
    print g[0]
    print
    print g[0].degree(x)
    explist=polynomialExponentList(D(g[0]))
    # explist = 1 => g[0] is irreducible.
    data=data+[explist]
print data

n:  2
(-8, 0)

-8

0
n:  3
(28*x^2 - 176*x + 304, 0)

28*x^2 - 176*x + 304

2
n:  4
(-64*x^4 + 2816/3*x^3 - 134144/27*x^2 + 332800/27*x - 336896/27, 0)

-64*x^4 + 2816/3*x^3 - 134144/27*x^2 + 332800/27*x - 336896/27

4
n:  5
(126*x^6 - 8440/3*x^5 + 728600/27*x^4 - 1258688/9*x^3 + 11223392/27*x^2 - 6282880/9*x + 14452352/27, 0)

126*x^6 - 8440/3*x^5 + 728600/27*x^4 - 1258688/9*x^3 + 11223392/27*x^2 - 6282880/9*x + 14452352/27

6
[[], [1], [1], [1]]


In [7]:
import pickle
rfile = open('/Users/barrybrent/run12aug21no2.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [2..len(s)-1]:
    n=s[k][0]
    poly=s[k][1]
    divideby=(x-2)^2*x^(n-1)
    g=division(poly,divideby)
    explist=polynomialExponentList(D(g[0]))
    # explist = 1 => g[0] is irreducible.
    data=data+[explist]
print data

[[], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1]]


In [12]:
import pickle
rfile = open('/Users/barrybrent/run12aug21no2.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [2..len(s)-1]:
    n=s[k][0]
    poly=D(s[k][1])
    divideby=(x-2)^2*x^(n-1)
    g=division(poly,divideby)
    dg=g[0].degree(x)
    # explist = 1 => g[0] is irreducible.
    data=data+[dg]
print data

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [19]:
import pickle
rfile = open('/Users/barrybrent/run12aug21no2.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [2..len(s)-1]:
    n=s[k][0]
    poly=s[k][1]
    divideby=(x-2)^2*x^(n-1)
    g=division(poly,divideby)
    d= g[0].degree(x)
    data=data+[(n,d)]
print data

[(2, 0), (3, 2), (4, 4), (5, 6), (6, 8), (7, 10), (8, 12), (9, 14), (10, 16), (11, 18), (12, 20), (13, 22), (14, 24), (15, 26), (16, 28), (17, 30), (18, 32), (19, 34), (20, 36), (21, 38), (22, 40), (23, 42), (24, 44), (25, 46), (26, 48), (27, 50), (28, 52), (29, 54), (30, 56), (31, 58), (32, 60), (33, 62), (34, 64), (35, 66), (36, 68), (37, 70), (38, 72), (39, 74), (40, 76), (41, 78), (42, 80), (43, 82), (44, 84), (45, 86), (46, 88), (47, 90), (48, 92), (49, 94), (50, 96), (51, 98), (52, 100), (53, 102), (54, 104), (55, 106), (56, 108), (57, 110), (58, 112), (59, 114), (60, 116), (61, 118), (62, 120), (63, 122), (64, 124), (65, 126), (66, 128), (67, 130), (68, 132), (69, 134), (70, 136), (71, 138), (72, 140), (73, 142), (74, 144), (75, 146), (76, 148), (77, 150), (78, 152), (79, 154), (80, 156), (81, 158), (82, 160), (83, 162), (84, 164), (85, 166), (86, 168), (87, 170), (88, 172), (89, 174), (90, 176), (91, 178), (92, 180), (93, 182), (94, 184), (95, 186), (96, 188), (97, 190), (98, 1

In [20]:
import pickle
rfile = open('/Users/barrybrent/run12aug21no2.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [2..len(s)-1]:
    n=s[k][0]
    poly=s[k][1]
    divideby=(x-2)^2*x^(n-1)
    g=division(poly,divideby)
    data=data+[g[1]]#remainders
print data

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [21]:
import pickle
rfile = open('/Users/barrybrent/run12aug21no2.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data1=[]
data2=[]
for k in [2..len(s)-1]:
    n=s[k][0]
    poly=s[k][1]
    divideby1=(x-2)^3
    g1=division(poly,divideby1)
    if g1[1]==0:data1=data1+[n]
    divideby2=x^n
    g2=division(poly,divideby2)
    if g2[1]==0:data1=data1+[n]
print data1;print data2

[]
[]
